In [27]:
import math

class Matrix:
    
    def __init__(self, data):
        if self.check_integrity(data) == False:
            raise Exception('Matrix is incorrectly configured with empty matrix or row, column, or data type mismatch. Please provide a list of lists containing only ints or floats of consistent dimensions.')
        self.data = data
        self.rows = len(data)
        self.cols = len(data[0])
        self.shape = (self.rows, self.cols)
        self.size = self.rows * self.cols

    def check_integrity(self, data) -> bool:
        '''
        Args - data (list): ...
        Returns - bool
        '''
        # Check if provided matrix is a list
        if isinstance(data, list) == False:
            return False
        
        # Check if provided matrix is a list of lists
        checks = [isinstance(x, list) == False for x in data]
        if True in checks:
            return False
        
        # Check if provided matrix has consistent dimensions
        ncols = [len(x) for x in data]
        if sum(ncols) / len(ncols) != ncols[0]:
            return False
        
        # Check if provided matrix has empty columns
        if ncols[0] == 0:
            return False
        
        # Check if provided matrix contains values of type int or float only
        for i in range(len(data)):
            checks = [isinstance(x, (int, float)) == False for x in data[i]]
            if sum(checks) > 0:
                return False

    def assign(self, row, col, value) -> None:
        if isinstance(value, (int, float)) == False:
            raise Exception('Value must be of type int or float only.')
        self.data[row][col] = value

    def fill(self, rows, cols, value) -> None:
        if isinstance(value, (int, float)) == False:
            raise Exception('Value must be of type int or float only.')
        for i in range(rows[0], rows[1], 1):
            for j in range(cols[0], cols[1], 1):
                self.assign(i, j, value)
    
    def retrieve(self, row, col) -> (int, float):
        return self.data[row][col]

    def diagonal(self) -> object:
        matrix = []
        for i in range(self.rows):
            if i < self.cols:
                matrix.append([self.data[i][i]])
        return Matrix(matrix)
    
    def magnitude(self) -> float:
        if not (self.shape[0] == 1 or self.shape[1] == 1):
            raise Exception('Matrix must be a row or column vector only.')
        matrix = []
        for i in range(self.rows):
            for j in range(self.cols):
                matrix.append(self.data[i][j]**2)
        return math.sqrt(sum(matrix))

    def transpose(self) -> object:
        matrix, row = [], []
        for j in range(self.cols):
            for i in range(self.rows):
                row.append(self.data[i][j])
            matrix.append(row)
            row = []
        return Matrix(matrix)

    # TODO
    def inverse(self) -> object:
        if self.shape[0] != self.shape[1] and self.shape[0] < 2:
            raise Exception('Matrix is not invertible as it is a non-square matrix. Please provide a square matrix with dimensions greater than or equal to (2, 2).')
        matrix = [[1], [0]]
        return Matrix(matrix)

    def determinant(self) -> float:
        if self.shape[0] != self.shape[1] and self.shape[0] < 2:
            raise Exception('Matrix determinant cannot be derived as it is a non-square matrix. Please provide a square matrix with dimensions greater than or equal to (2, 2).')

        matrix, data = self.data, self.data
        indices = list(range(len(matrix)))
        det = 0

        if len(matrix) == 2 and len(matrix[0]) == 2:
            det = matrix[0][0] * matrix[1][1] - matrix[1][0] * matrix[0][1]
            return det
        
        for focus_column in indices:
            matrix.remove(matrix[0])
            rows = len(matrix)
            for i in range(rows):
                matrix[i] = matrix[i][0:focus_column].extend(matrix[i][focus_column+1:])

        sub_det = Matrix(matrix).determinant()
        det += (-1)**(focus_column % 2) * matrix[0][focus_column] * sub_det

        return det

    def __add__(self, other) -> object:
        if other.shape != self.shape:
            raise Exception('Matrix dimensions are mismatched ({} != {}). Please add (2) matrices of the same dimensions.'.format(self.shape, other.shape))
        data, output = self.data, []
        for i in range(self.rows):
            output.append([])
            for j in range(self.cols):
                output[i].append(self.data[i][j] + other.data[i][j])
        self.data = data
        return Matrix(output)
    
    def __ladd__(self, other) -> object:
        return self.__add__(other)

    def __radd__(self, other) -> object:
        return self.__add__(other)

    def __sub__(self, other) -> object:
        if other.shape != self.shape:
            raise Exception('Matrix dimensions are mismatched ({} != {}). Please subtract (2) matrices of the same dimensions.'.format(self.shape, other.shape))
        data, output = self.data, []
        for i in range(self.rows):
            output.append([])
            for j in range(self.cols):
                output[i].append(self.data[i][j] - other.data[i][j])
        self.data = data
        return Matrix(output)
    
    def __lsub__(self, other) -> object:
        return self.__sub__(other)

    def __rsub__(self, other) -> object:
        return self.__sub__(other)

    def __mul__(self, other) -> object:
        if isinstance(other, Matrix) == False:
            if isinstance(other, (int, float)) == False:
                raise Exception('Expecting a matrix or a scalar of type int or float only.')
        else:
            if self.size != other.size:
                raise Exception('Matrix dimensions are mismatched for pointwise matrix multiplication ({} != {})'.format(self.shape, other.shape))

        data, output = self.data, []
        
        # Scalar multiplication
        if isinstance(other, (int, float)):
            for i in range(self.rows):
                output.append([])
                for j in range(self.cols):
                    output[i].append(self.data[i][j] * other)

        # Matrix multiplication (pointwise)
        if isinstance(other, Matrix):
            for i in range(self.rows):
                output.append([])
                for j in range(self.cols):
                    output[i].append(self.data[i][j] * other.data[i][j])

        self.data = data
        return Matrix(output)

    def __lmul__(self, other) -> object:
        return self.__mul__(other)

    def __rmul__(self, other) -> object:
        return self.__mul__(other)

    def __matmul__(self, other) -> object:
        if self.cols != other.rows:
            raise Exception('Matrix dimensions are mismatched for matrix multiplication ({} @ {}: {} != {})'.format(self.shape, other.shape, self.shape[1], other.shape[0]))
        data, output = self.data, []
        n, m = self.shape[1], self.shape[1]
        for i in range(n):
            output.append([])
            for j in range(m):
                value = 0
                for k in range(n):
                    value += self.data[i][k] * other.data[k][j]
                output[i].append(value)
        self.data = data
        return Matrix(output)
    
    def __lmatmul__(self, other) -> object:
        return __matmul__(self, other)
    
    def __rmatmul__(self, other) -> object:
        return __matmul__(self, other)

    def __repr__(self) -> str:
        matrix = str(self.data).replace('],', '],\n ')
        return "<class: 'Matrix'>\nDimensions: {} row(s) x {} column(s)\n {}\nSize: {}".format(self.rows, self.cols, matrix, self.size)

In [28]:
def determinant_recursive(A, total=0):
    # Section 1: store indices in list for row referencing
    indices = list(range(len(A)))
     
    # Section 2: when at 2x2 submatrices recursive calls end
    if len(A) == 2 and len(A[0]) == 2:
        val = A[0][0] * A[1][1] - A[1][0] * A[0][1]
        return val
 
    # Section 3: define submatrix for focus column and 
    #      call this function
    for fc in indices: # A) for each focus column, ...
        # find the submatrix ...
        As = copy_matrix(A) # B) make a copy, and ...
        As = As[1:] # ... C) remove the first row
        height = len(As) # D) 
 
        for i in range(height): 
            # E) for each remaining row of submatrix ...
            #     remove the focus column elements
            As[i] = As[i][0:fc] + As[i][fc+1:] 
 
        sign = (-1) ** (fc % 2) # F) 
        # G) pass submatrix recursively
        sub_det = determinant_recursive(As)
        # H) total all returns from recursion
        total += sign * A[0][fc] * sub_det 
 
    return total

In [29]:
A = Matrix([[1, 0, 0, 0], [0, 2, 2, 0], [0, 0, 3, 0], [0, 0, 0, 4]])

In [30]:
A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[1, 0, 0, 0],
  [0, 2, 2, 0],
  [0, 0, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [31]:
A.fill((0, 2), (0, 2), 5)

In [32]:
A.magnitude()

Exception: Matrix must be a row or column vector only.

In [33]:
print(A.shape)
print(A.size)

(4, 4)
16


In [34]:
A.transpose()

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[5, 5, 0, 0],
  [5, 5, 0, 0],
  [0, 2, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [35]:
A+A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[10, 10, 0, 0],
  [10, 10, 4, 0],
  [0, 0, 6, 0],
  [0, 0, 0, 8]]
Size: 16

In [36]:
A-A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]]
Size: 16

In [37]:
A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[5, 5, 0, 0],
  [5, 5, 2, 0],
  [0, 0, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [38]:
A * 2

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[10, 10, 0, 0],
  [10, 10, 4, 0],
  [0, 0, 6, 0],
  [0, 0, 0, 8]]
Size: 16

In [39]:
A.transpose() * 2

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[10, 10, 0, 0],
  [10, 10, 0, 0],
  [0, 4, 6, 0],
  [0, 0, 0, 8]]
Size: 16

In [40]:
2 * A - A + 3 * A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[20, 20, 0, 0],
  [20, 20, 8, 0],
  [0, 0, 12, 0],
  [0, 0, 0, 16]]
Size: 16

In [41]:
A * A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[25, 25, 0, 0],
  [25, 25, 4, 0],
  [0, 0, 9, 0],
  [0, 0, 0, 16]]
Size: 16

In [42]:
A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[5, 5, 0, 0],
  [5, 5, 2, 0],
  [0, 0, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [43]:
A.diagonal()

<class: 'Matrix'>
Dimensions: 4 row(s) x 1 column(s)
 [[5],
  [5],
  [3],
  [4]]
Size: 4

In [44]:
B = Matrix([[1], [0]])

In [45]:
B

<class: 'Matrix'>
Dimensions: 2 row(s) x 1 column(s)
 [[1],
  [0]]
Size: 2

In [46]:
A @ A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[50, 50, 10, 0],
  [50, 50, 16, 0],
  [0, 0, 9, 0],
  [0, 0, 0, 16]]
Size: 16

In [47]:
A @ B

Exception: Matrix dimensions are mismatched for matrix multiplication ((4, 4) @ (2, 1): 4 != 2)

In [48]:
B.diagonal()

<class: 'Matrix'>
Dimensions: 1 row(s) x 1 column(s)
 [[1]]
Size: 1

In [49]:
B.assign(0, 0, 5)

In [50]:
B

<class: 'Matrix'>
Dimensions: 2 row(s) x 1 column(s)
 [[5],
  [0]]
Size: 2

In [51]:
B.magnitude()

5.0

In [53]:
A.fill((0, 4), (0, 1), 7)

In [54]:
A.retrieve(2, 2)

3

In [55]:
A+B

Exception: Matrix dimensions are mismatched ((4, 4) != (2, 1)). Please add (2) matrices of the same dimensions.

In [56]:
C = Matrix([[]])

Exception: Matrix is incorrectly configured with empty matrix or row, column, or data type mismatch. Please provide a list of lists containing only ints or floats of consistent dimensions.

In [58]:
A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[7, 5, 0, 0],
  [7, 5, 2, 0],
  [7, 0, 3, 0],
  [7, 0, 0, 4]]
Size: 16